In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#download traning data from open datasets

training_data = datasets.FashionMNIST(
    root = 'data', 
    train= True,
    download= True,
    transform= ToTensor()
)

#download test data from open datasets
test_data = datasets.FashionMNIST(
    root= 'data',
    train= False,
    download= True,
    transform= ToTensor()
)

100%|██████████| 26.4M/26.4M [00:27<00:00, 949kB/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 129kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:11<00:00, 383kB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 1.27MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size= batch_size)

for X, y in test_dataloader:
    print(f'shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape o

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)

model

using cpu device


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>f} [{current:>5d}/{size:>5d}]')

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size
        print(f'test error: \n accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [10]:
epochs = 5
for t in range(epochs):
    print(f'epoch {t + 1}: \n ---------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("done!")

epoch 1: 
 ---------------------------
loss: 2.294078 [   64/60000]
loss: 2.277726 [ 6464/60000]
loss: 2.262486 [12864/60000]
loss: 2.258634 [19264/60000]
loss: 2.242288 [25664/60000]
loss: 2.214367 [32064/60000]
loss: 2.228816 [38464/60000]
loss: 2.193211 [44864/60000]
loss: 2.186557 [51264/60000]
loss: 2.162174 [57664/60000]
test error: 
 accuracy: 40.3%, Avg loss: 2.151304 

epoch 2: 
 ---------------------------
loss: 2.163598 [   64/60000]
loss: 2.141874 [ 6464/60000]
loss: 2.094150 [12864/60000]
loss: 2.114731 [19264/60000]
loss: 2.054255 [25664/60000]
loss: 2.003606 [32064/60000]
loss: 2.039316 [38464/60000]
loss: 1.957105 [44864/60000]
loss: 1.967373 [51264/60000]
loss: 1.900853 [57664/60000]
test error: 
 accuracy: 51.9%, Avg loss: 1.891144 

epoch 3: 
 ---------------------------
loss: 1.926140 [   64/60000]
loss: 1.881231 [ 6464/60000]
loss: 1.779591 [12864/60000]
loss: 1.828485 [19264/60000]
loss: 1.703170 [25664/60000]
loss: 1.672295 [32064/60000]
loss: 1.705179 [38464/600

In [11]:
torch.save(model.state_dict(), 'model.pth')
print("Saved!")

Saved!


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))


<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

x, y = test_data[0][0], test_data[0][1]
# print(x, y)
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(pred)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'predicted: {predicted}, actual: {actual}')

tensor([[-2.2666, -2.6959, -0.8362, -2.2123, -0.9235,  2.4797, -1.0543,  2.5949,
          1.8014,  3.1702]])
predicted: Ankle boot, actual: Ankle boot
